# Syrtis Demonstration
# Heat Gain into a Cryogen Tank at Boca Chica, TX

To demonstrate the StaticShell feature of Syrtis, we can analyse a simple system. SpaceX have made use of large double-walled steel tanks to store huge volumes of cryogenic propellants at their Boca Chica facility. These have an inner wall diameter of around 9m (to allow common tooling with the Starship/Super Heavy vehicle) and an outer wall diameter of around 12m. The gap between them is believed to be filled with expanded perlite.

The choice to use perlite rather than a simple air gap, as well as the dimensions of this gap, can be crudely analysed with Syrtis. It should be noted that this is not an intended use for the solver, as it uses the enormously dense terrestrial atmosphere, but serves as an interesting demonstration.

In [3]:
from syrtis import *
from syrtis.configuration import Configuration
from syrtis.solver import Solver

In [4]:
steel = Solid("Steel", 150, 8700, 500, 0.55)
co2 = ConstrainedIdealGas("STP CO2", 101325, 44, 0.71, 10.9e-6, 749, 0.0153)

bocachica = Configuration("bocachica", "constant temperature",
300, 1, 0.29, 300, 101325, 1, "cross", 90, 90, 1000, T_habitat=80)

heat_gain = []
thicknesses = np.logspace(-3, 0, 20)
#thicknesses = np.linspace(0.001, 1.001, 500)

for thickness in thicknesses:
    tankfarm = Habitat("vertical", 50, "flat")
    tankfarm.create_static_shell(co2, 4.5)
    tankfarm.create_static_shell(steel, 0.001)
    tankfarm.create_static_shell(co2, thickness)
    tankfarm.create_static_shell(steel, 0.001)

    tankfarm.verify_geometry()

    s = Solver("test", tankfarm, bocachica)
    q = s.iterate_constant_temperature()

    heat_gain.append(-q)

import matplotlib.pyplot as plt

plt.scatter(thicknesses, heat_gain)
plt.xscale("log")
plt.xlim(9e-4, 1.5)
plt.xlabel("Gap between inner and outer wall (m)")
plt.ylabel("Heat gain into tank (W)")
plt.title("Syrtis evaluation case \n Heat gain into GSE tanks at Boca Chica tank farm")
plt.show()

NameError: name 'Habitat' is not defined